- load namelist
- train model(s) male/female
- filter results for "unkown" names
- store results
- (batch) send to msg-api

In [56]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from IPython.display import clear_output
import collections
import urllib
from os import listdir
from os.path import isfile, join
import pandas
import re

In [138]:
#source: https://daten.berlin.de/datensaetze/liste-der-h%C3%A4ufigen-vornamen-2015

fileurllist = ['http://www.berlin.de/daten/liste-der-vornamen-2015/charlottenburg-wilmersdorf.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/friedrichshain-kreuzberg.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/lichtenberg.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/marzahn-hellersdorf.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/mitte.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/neukoelln.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/pankow.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/reinickendorf.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/spandau.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/steglitz-zehlendorf.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/tempelhof-schoeneberg.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/treptow-koepenick.csv']
!mkdir name-csvs/
for url in fileurllist:
    file_name = url.split('/')[-1]
    full_path = 'name-csvs/' + file_name
    urllib.urlretrieve(url, full_path)

Build a proper text file for training

In [149]:
def build_training_data(gender="w", districts="all", weighted=False):
    # returns one string, names separated by \n, names randomized, repetitions across districts NOT removed
    raw_names = ''
    if (districts == "all"):
        print "loading all districts (in folder)"
        
        files_in_dir = [f for f in listdir('name-csvs/') if isfile(join('name-csvs/', f))]
        #print files_in_dir
        
        for current_file in files_in_dir:
            #print join('name-csvs/', current_file)
            temp_df = pandas.read_csv(join('name-csvs/', current_file),';')
            #print(temp_df.head())
            #print len(temp_df)
            #print 'Adding ' + str(len(temp_df)) + ' names from file ' + current_file
            temp_str = re.sub(pattern = r' |\d',repl='', string = temp_df.reindex(np.random.permutation(temp_df.index)).loc[temp_df['geschlecht'] == gender].vorname.to_string())
            raw_names += temp_str
            
    else:#better check if is list
        print "loading districts as specified in list: " + str(districts)
          
        for current_file in districts:

            temp_df = pandas.read_csv(join('name-csvs/', current_file),';')
            temp_str = re.sub(pattern = r' |\d',repl='', string = temp_df.reindex(np.random.permutation(temp_df.index)).loc[temp_df['geschlecht'] == "w"].vorname.to_string())
            raw_names += temp_str
    
    return raw_names

In [145]:
my_districts = ['lichtenberg.csv', 'marzahn-hellersdorf.csv', 'neukoelln.csv', 'reinickendorf.csv']

In [144]:
names_w_all_districts = build_training_data()

loading all districts (in folder)


In [150]:
names_w_my_districts = build_training_data(districts= my_districts)

loading districts as specified in list: ['lichtenberg.csv', 'marzahn-hellersdorf.csv', 'neukoelln.csv', 'reinickendorf.csv']
